# 准备数据

In [1]:
import sys

sys.path.append('../')

from alignment_utils import one_file_per_process
from alignment_utils import extract_and_combine_numbers
from alignment_utils import longest_increasing_subsequence_index
from alignment_utils import find_answer_split_str
from alignment_utils import find_next_question_index
from alignment_utils import refine_answers
from alignment_utils import match_specific_from_end
from alignment_utils import answer_area_str_process
from alignment_utils import generate_answer_area_string
from alignment_utils import align_answers_in_questions

In [2]:
import os
import glob
from pathlib import Path

examination_paper_list = []

path = Path("./answer_markdown")

for file in path.glob("*.md"):
    with open(file, "r", encoding="utf-8") as f:
        examination_paper_list.append({
            "file_path": file,
            "text": one_file_per_process(f.read())
        })

# 寻找题目

In [3]:
def get_all_question(text):
    # 拆分成行
    lines = text.splitlines()
    
    # 定义不准确的题目列表    
    inaccuracy_question = []
    
    # 从0的位置寻找第一道题
    index = find_next_question_index(0, lines)
    
    while index < len(lines):
        # 寻找下一个题目的index
        next_index = find_next_question_index(index, lines)
        
        inaccuracy_question.append("\n".join(lines[index: next_index]))
        index = next_index
#     for i in inaccuracy_question:
#         print(i)
#         print()
    print( [extract_and_combine_numbers(topic) for topic in inaccuracy_question])
    # 通过"最长递增子序列"寻找每个精准的题目所在inaccuracy_question对应的下标
    all_question_indexs = longest_increasing_subsequence_index(inaccuracy_question)
    
    # 定义准确的题目列表    
    all_question = []
    # index为all_question_indexs的下标，all_question_indexs[index]为inaccuracy_question的下标
    for index, question_index in enumerate(all_question_indexs):
        if index == len(all_question_indexs) -1 :
            all_question.append("\n".join(inaccuracy_question[question_index:]))
            break
            
        all_question.append("\n".join(inaccuracy_question[question_index:all_question_indexs[index+1]]))
    
    if not all_question:
        return None, None
    
    
    # 尝试寻找用于分割答题区与答案区的字符串，返回值为int/str，如果是str则是分割的字符串
    # 本质是在"all_question[-1]"寻找答案关键字等字样
    answer_split_str = find_answer_split_str(all_question)
    
    if isinstance(answer_split_str, str):
        # 如果找到这个拆分的字符串了，则先把最后一道题的内容进行拆分
        all_question[-1] = all_question[-1].split(answer_split_str)[0]
        
    if text.splitlines()[0] in all_question[-1]:
        answer_split_str = text.splitlines()[0]
        # 看看试卷的title是否出现在"all_question[-1]"位置，如果出现则删除
        all_question[-1] = all_question[-1].replace("\n"+text.splitlines()[0]+"\n","")

    return all_question, answer_split_str

# 创建测试样本

In [4]:
import random

test_text = examination_paper_list[random.randint(0, len(examination_paper_list))]["text"]

In [5]:
test_text.splitlines()

['2019年黑龙江省龙东地区中考数学试卷',
 '一、填空题（每题3分，满分30分）',
 '1．（3分）（2019•黑龙江）中国政府提出的"一带一路"倡议，近两年来为沿线国家创造了约180000个就业岗位．将数据180000用科学记数法表示为[]{.underline}．',
 '2．（3分）（2019•黑龙江）在函数中，自变量的取值范围是[]{.underline}．',
 '3．（3分）（2019•黑龙江）如图，在四边形中，，在不添加任何辅助线的情况下，请你添加一个条件[]{.underline}，使四边形是平行四边形．',
 '![](./notebook/image/media/image6.png)',
 '4．（3分）（2019•黑龙江）在不透明的甲、乙两个盒子中装有除颜色外完全相同的小球，甲盒中有2个白球、1个黄球，乙盒中有1个白球、1个黄球，分别从每个盒中随机摸出1个球，则摸出的2个球都是黄球的概率是[]{.underline}．',
 '5．（3分）（2019•黑龙江）若关于的一元一次不等式组的解集为，则的取值范围是[]{.underline}．',
 '6．（3分）（2019•黑龙江）如图，在中，半径垂直于弦，点在圆上且，则的度数为[]{.underline}．',
 '![](./notebook/image/media/image17.png)',
 '7．（3分）（2019•黑龙江）若一个圆锥的底面圆的周长是，母线长是，则该圆锥的侧面展开图的圆心角度数是[]{.underline}．',
 '8．（3分）（2019•黑龙江）如图，矩形中，，，点是矩形内一动点，且，则的最小值为[]{.underline}．',
 '![](./notebook/image/media/image27.png)',
 '9．（3分）（2019•黑龙江）一张直角三角形纸片，，，，点为边上的任一点，沿过点的直线折叠，使直角顶点落在斜边上的点处，当是直角三角形时，则的长为[]{.underline}．',
 '10．（3分）（2019•黑龙江）如图，四边形是边长为1的正方形，以对角线为边作第二个正方形，连接，得到△；再以对角线为边作第三个正方形，连接，得到△；再以对角线为边作第四个正方形，连接，得到△记△、△、△的面积分别为、、，如此下去，则[]{.underl

In [6]:
# split_str用于进行下一步的分析（有可能这份试卷是有答题区和答案区，也有可能每道题的答案都在题的下方，还有可能根本没有答案）
all_question, split_str = get_all_question(test_text)
print(f"split_str:\n{split_str}")
if split_str in [-1, 0]:
    print("此试卷无答案")
all_question

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 1, 2, 3, 23, 1, 2, 24, 1, 2, 3, 4, 25, 1, 2, 3, 26, 1, 2, 27, 1, 2, 3, 28, 1, 2, 3, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 1, 2, 3, 2, 3, 2, 3, 23, 1, 2, 2, 2, 24, 1, 2, 3, 4, 2, 3, 4, 2, 3, 4, 25, 1, 2, 3, 2, 3, 2, 3, 26, 1, 2, 2, 2, 27, 1, 2, 3, 2, 3, 2, 3, 28, 1, 2, 3, 2, 3, 2, 3]
split_str:
2019年黑龙江省龙东地区中考数学试卷


['1．（3分）（2019•黑龙江）中国政府提出的"一带一路"倡议，近两年来为沿线国家创造了约180000个就业岗位．将数据180000用科学记数法表示为[]{.underline}．',
 '2．（3分）（2019•黑龙江）在函数中，自变量的取值范围是[]{.underline}．',
 '3．（3分）（2019•黑龙江）如图，在四边形中，，在不添加任何辅助线的情况下，请你添加一个条件[]{.underline}，使四边形是平行四边形．\n![](./notebook/image/media/image6.png)',
 '4．（3分）（2019•黑龙江）在不透明的甲、乙两个盒子中装有除颜色外完全相同的小球，甲盒中有2个白球、1个黄球，乙盒中有1个白球、1个黄球，分别从每个盒中随机摸出1个球，则摸出的2个球都是黄球的概率是[]{.underline}．',
 '5．（3分）（2019•黑龙江）若关于的一元一次不等式组的解集为，则的取值范围是[]{.underline}．',
 '6．（3分）（2019•黑龙江）如图，在中，半径垂直于弦，点在圆上且，则的度数为[]{.underline}．\n![](./notebook/image/media/image17.png)',
 '7．（3分）（2019•黑龙江）若一个圆锥的底面圆的周长是，母线长是，则该圆锥的侧面展开图的圆心角度数是[]{.underline}．',
 '8．（3分）（2019•黑龙江）如图，矩形中，，，点是矩形内一动点，且，则的最小值为[]{.underline}．\n![](./notebook/image/media/image27.png)',
 '9．（3分）（2019•黑龙江）一张直角三角形纸片，，，，点为边上的任一点，沿过点的直线折叠，使直角顶点落在斜边上的点处，当是直角三角形时，则的长为[]{.underline}．',
 '10．（3分）（2019•黑龙江）如图，四边形是边长为1的正方形，以对角线为边作第二个正方形，连接，得到△；再以对角线为边作第三个正方形，连接，得到△；再以对角线为边作第四个正方形，连接，得到△记△、△、△的面积分别为、、，如此下去，则[]{.underline}．\n![](./notebook/image/media/image59.png)\n二、选择题（每

In [122]:
if split_str == 1:
    for row in align_answers_in_questions(all_question):
        print()
        print(row["question"])
        print("-------------------------------------------------------")
        print(f"{row['answer']}")
        print()
        print("============================================================")

# 寻找答案（试卷存在答题区与答案区）

In [7]:
def get_all_answer_sequence(answer_area_string):
    lines = answer_area_string.splitlines()
    
    inaccuracy_answers = []

    index = find_next_question_index(0, lines)
    while index < len(lines):
#         print(lines[index])
#         print()
        next_index = find_next_question_index(index, lines)
       
        inaccuracy_answers.append("\n".join(lines[index: next_index]))
        index = next_index
#     print("============")
#     for i in inaccuracy_answers:
#         print(i)
#         print()
        
#     print([extract_and_combine_numbers(topic) for topic in inaccuracy_answers])
    inaccuracy_answer_indexes = longest_increasing_subsequence_index(inaccuracy_answers)
   
    processed_inaccuracy_answers = []
    for index, answer_index in enumerate(inaccuracy_answer_indexes):
        if index == len(inaccuracy_answer_indexes) -1 :
            processed_inaccuracy_answers.append(inaccuracy_answers[answer_index])
            break
        processed_inaccuracy_answers.append("\n".join(inaccuracy_answers[answer_index:inaccuracy_answer_indexes[index+1]]))
        
    return refine_answers(processed_inaccuracy_answers)[::-1]

In [8]:
answer_str = generate_answer_area_string(test_text, split_str)

In [9]:
answer_str.splitlines()

['',
 '参考答案与试题解析',
 '一、填空题（每题3分，满分30分）',
 '1．（3分）中国政府提出的"一带一路"倡议，近两年来为沿线国家创造了约180000个就业岗位．将数据180000用科学记数法表示为[]{.underline}．',
 '【考点】科学记数法表示较大的数',
 '【分析】科学记数法的表示形式为的形式，其中，为整数．确定的值时，要看把原数变成时，小数点移动了多少位，的绝对值与小数点移动的位数相同．当原数绝对值时，是正数；当原数的绝对值时，是负数．',
 '【解答】解：将180000用科学记数法表示为，',
 '故答案是：．',
 '2．（3分）在函数中，自变量的取值范围是[]{.underline}．',
 '【考点】函数自变量的取值范围',
 '【分析】根据二次根式有意义的条件是被开方数大于或等于0即可求解．',
 '【解答】解：在函数中，有，解得，',
 '故其自变量的取值范围是．',
 '故答案为．',
 '3．（3分）如图，在四边形中，，在不添加任何辅助线的情况下，请你添加一个条件[（答案不唯一）]{.underline}，使四边形是平行四边形．',
 '![](./notebook/image/media/image265.png)',
 '【考点】平行四边形的判定',
 '【分析】可再添加一个条件，根据两组对边分别相等的四边形是平行四边形，四边形是平行四边形．',
 '【解答】解：根据平行四边形的判定，可再添加一个条件：．',
 '故答案为：（答案不唯一）．',
 '4．（3分）在不透明的甲、乙两个盒子中装有除颜色外完全相同的小球，甲盒中有2个白球、1个黄球，乙盒中有1个白球、1个黄球，分别从每个盒中随机摸出1个球，则摸出的2个球都是黄球的概率是[]{.underline}．',
 '【考点】列表法与树状图法',
 '【分析】先画出树状图展示所有6种等可能的结果数，再找出2个球都是黄球所占结果数，然后根据概率公式求解．',
 '【解答】解：画树状图为：，',
 '共有6种等可能的结果数，其中2个球都是黄球占1种，',
 '摸出的2个球都是黄球的概率；',
 '故答案为：．',
 '![](./notebook/image/media/image274.png)',
 '5．（3分）若关于的一元一次不等式组的解集为，则的取值范

In [10]:
for i in get_all_answer_sequence(answer_area_str_process(answer_str)):
    print(i)
    print()

1．（3分）中国政府提出的"一带一路"倡议，近两年来为沿线国家创造了约180000个就业岗位．将数据180000用科学记数法表示为[]{.underline}．
【考点】科学记数法表示较大的数
【分析】科学记数法的表示形式为的形式，其中，为整数．确定的值时，要看把原数变成时，小数点移动了多少位，的绝对值与小数点移动的位数相同．当原数绝对值时，是正数；当原数的绝对值时，是负数．
【解答】解：将180000用科学记数法表示为，
故答案是：．

2．（3分）在函数中，自变量的取值范围是[]{.underline}．
【考点】函数自变量的取值范围
【分析】根据二次根式有意义的条件是被开方数大于或等于0即可求解．
【解答】解：在函数中，有，解得，
故其自变量的取值范围是．
故答案为．

3．（3分）如图，在四边形中，，在不添加任何辅助线的情况下，请你添加一个条件[（答案不唯一）]{.underline}，使四边形是平行四边形．
![](./notebook/image/media/image265.png)
【考点】平行四边形的判定
【分析】可再添加一个条件，根据两组对边分别相等的四边形是平行四边形，四边形是平行四边形．
【解答】解：根据平行四边形的判定，可再添加一个条件：．
故答案为：（答案不唯一）．

4．（3分）在不透明的甲、乙两个盒子中装有除颜色外完全相同的小球，甲盒中有2个白球、1个黄球，乙盒中有1个白球、1个黄球，分别从每个盒中随机摸出1个球，则摸出的2个球都是黄球的概率是[]{.underline}．
【考点】列表法与树状图法
【分析】先画出树状图展示所有6种等可能的结果数，再找出2个球都是黄球所占结果数，然后根据概率公式求解．
【解答】解：画树状图为：，
共有6种等可能的结果数，其中2个球都是黄球占1种，
摸出的2个球都是黄球的概率；
故答案为：．
![](./notebook/image/media/image274.png)

5．（3分）若关于的一元一次不等式组的解集为，则的取值范围是[]{.underline}．
【考点】解一元一次不等式组
【分析】分别求出每一个不等式的解集，根据口诀：同大取大、同小取小、大小小大中间找、大大小小无解了确定不等式组的解集．
【解答】解：解不等式，得：，
解不等式，得：，
不等式组的解集为，
，
故答案为：．

6．（3

# 对齐

In [11]:
def alignment_answer(all_question, answer_str):
    questions_with_answer = []
    all_answer = get_all_answer_sequence(answer_area_str_process(answer_str))

    questions_map = {extract_and_combine_numbers(question):question for question in all_question} 
    answer_map = {extract_and_combine_numbers(answer):answer for answer in all_answer}
    
    for sequence_number in questions_map:
        questions_with_answer.append({"question":questions_map.get(sequence_number),"answer":answer_map.get(sequence_number, None)})
    return questions_with_answer

In [12]:
for row in alignment_answer(all_question, answer_str):
    print()
    print(row["question"])
    print("-------------------------------------------------------")
    print(f"{row['answer']}")
    print()
    print("============================================================")


1．（3分）（2019•黑龙江）中国政府提出的"一带一路"倡议，近两年来为沿线国家创造了约180000个就业岗位．将数据180000用科学记数法表示为[]{.underline}．
-------------------------------------------------------
1．（3分）中国政府提出的"一带一路"倡议，近两年来为沿线国家创造了约180000个就业岗位．将数据180000用科学记数法表示为[]{.underline}．
【考点】科学记数法表示较大的数
【分析】科学记数法的表示形式为的形式，其中，为整数．确定的值时，要看把原数变成时，小数点移动了多少位，的绝对值与小数点移动的位数相同．当原数绝对值时，是正数；当原数的绝对值时，是负数．
【解答】解：将180000用科学记数法表示为，
故答案是：．


2．（3分）（2019•黑龙江）在函数中，自变量的取值范围是[]{.underline}．
-------------------------------------------------------
2．（3分）在函数中，自变量的取值范围是[]{.underline}．
【考点】函数自变量的取值范围
【分析】根据二次根式有意义的条件是被开方数大于或等于0即可求解．
【解答】解：在函数中，有，解得，
故其自变量的取值范围是．
故答案为．


3．（3分）（2019•黑龙江）如图，在四边形中，，在不添加任何辅助线的情况下，请你添加一个条件[]{.underline}，使四边形是平行四边形．
![](./notebook/image/media/image6.png)
-------------------------------------------------------
3．（3分）如图，在四边形中，，在不添加任何辅助线的情况下，请你添加一个条件[（答案不唯一）]{.underline}，使四边形是平行四边形．
![](./notebook/image/media/image265.png)
【考点】平行四边形的判定
【分析】可再添加一个条件，根据两组对边分别相等的四边形是平行四边形，四边形是平行四边形．
【解答】解：根据平行四边形的判定，可再添加一个条件：．
故答案为：（答案不唯一）．


4．（3分）（2019•黑龙江）在不透明

# 生产测试

In [88]:
import pandas as pd
from tqdm import tqdm

# 创建一个空的 DataFrame
df = pd.DataFrame()

# 无答案的文件名列表
failed_files = []

for examination_paper in tqdm(examination_paper_list):
    try:
        file_path = str(examination_paper["file_path"])
        file_name = file_path[file_path.rfind("/")+1:]

        text = examination_paper["text"]
        all_question, split_str = get_all_question(text)

        if split_str in [-1, 0]:
#             print(f"{file_name} 试卷无答案")
            failed_files.append(file_name)
            continue

        # question_with_answer like is [{"question":str, "answer":str}]
        if split_str == 1:
            question_with_answer = align_answers_in_questions(all_question)
        else:
            answer_str = generate_answer_area_string(test_text, split_str)
            question_with_answer = alignment_answer(all_question, split_str)

        # 创建一个临时的 DataFrame
        temp_df = pd.DataFrame(question_with_answer)

        # 添加文件名列
        temp_df['file_name'] = file_name

        # 使用 pd.concat 将 temp_df 添加到主 DataFrame 中
        df = pd.concat([df, temp_df], ignore_index=True)
    except:
        failed_files.append(file_name)
        continue

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1615/1615 [00:01<00:00, 853.42it/s]


In [92]:
df[df['answer'].notna()]

,question,answer,file_name
160,1.的绝对值是（ ）\nA. B. 2 C. D.,【答案】B\n【解析】\n【分析】\n根据负数的绝对值是它的相反数，可得一个数的绝对值．\n...,湖北襄阳-word解析.md
161,2.如图，，直线分别交，于点E，F，平分，若，则的大小是（ ）\n![](./noteboo...,【答案】C\n【解析】\n【分析】\n利用平行线的性质求解，利用角平分线求解，再利用平行线的...,湖北襄阳-word解析.md
162,3.下列运算一定正确的是（ ）\nA. B. C. D.,【答案】C\n【解析】\n【分析】\n利用合并同类项，同底数幂乘法，幂的乘方与积的乘方运算法...,湖北襄阳-word解析.md
163,"4.下列说法正确的是（ ）\nA. ""买中奖率为的奖券10张，中奖""是必然事件\nB. ""汽...",【答案】D\n【解析】\n【分析】\n根据事件发生的可能性大小判断相应事件的类型，以及方差的...,湖北襄阳-word解析.md
164,5.如图所示的三视图表示的几何体是（ ）\n![](./notebook/image/med...,【答案】A\n【解析】\n![](./notebook/image/media/image4...,湖北襄阳-word解析.md
...,...,...,...
16349,(9),"【答案】.\n【解析】该常系数线性齐次微分方程的特征方程为 ,因式分解得\n,\n解得特征根...",2010考研数学二真题及答案解析.md
16350,(11),"【答案】.\n【解析】由高阶导数公式可知,\n所以 ,\n即..\n【解析】由高阶导数公式可...",2010考研数学二真题及答案解析.md
16351,(12),"【答案】.\n【解析】因为 ,所以对数螺线的极坐标弧长公式为\n==..\n【解析】因为 ,...",2010考研数学二真题及答案解析.md
16352,(13),"【答案】3.\n【解析】设,由题意知,在时刻,且 ,设该对角线长为,则 ,所以\n.\n所以...",2010考研数学二真题及答案解析.md


In [90]:
for index, row in df[df['answer'].notna()].iloc[1000:2000].iterrows():
    print()
    print(row["question"])
    print("-------------------------------------------------------")
    print(f"{row['answer']}")
    print()
    print("============================================================")


2.紫花前胡醇![](./notebook/image/media/image4.png)可从中药材当归和白芷中提取得到，能提高人体免疫力。有关该化合物，下列叙述错误的是
A. 分子式为C~14~H~14~O~4~
B. 不能使酸性重铬酸钾溶液变色
C. 能够发生水解反应
D. 能够发生消去反应生成双键
-------------------------------------------------------
【答案】B
【解析】
【详解】A.根据该有机物的分子结构可以确定其分子式为C~14~H~14~O~4~，A叙述正确；
B.该有机物的分子在有羟基，且与羟基相连的碳原子上有氢原子，故其可以被酸性重铬酸钾溶液氧化，能使酸性重铬酸钾溶液变色，B叙述不正确；
C.该有机物的分子中有酯基，故其能够发生水解反应，C叙述正确；
D.该有机物分子中与羟基相连的碳原子的邻位碳原子上有氢原子，故其可以在一定的条件下发生消去反应生成碳碳双键，D叙述正确。
综上所述，故选B。B
【解析】
【详解】A.根据该有机物的分子结构可以确定其分子式为C~14~H~14~O~4~，A叙述正确；
B.该有机物的分子在有羟基，且与羟基相连的碳原子上有氢原子，故其可以被酸性重铬酸钾溶液氧化，能使酸性重铬酸钾溶液变色，B叙述不正确；
C.该有机物的分子中有酯基，故其能够发生水解反应，C叙述正确；
D.该有机物分子中与羟基相连的碳原子的邻位碳原子上有氢原子，故其可以在一定的条件下发生消去反应生成碳碳双键，D叙述正确。
综上所述，故选B。


3.下列气体去除杂质的方法中，不能实现目的的是
  --- -------------- ----------------------
      气体(杂质)     方法
  A   SO~2~(H~2~S)   通过酸性高锰酸钾溶液
  B   Cl~2~(HCl)     通过饱和的食盐水
  C   N~2~(O~2~)     通过灼热的铜丝网
  D   NO(NO~2~)      通过氢氧化钠溶液
  --- -------------- ----------------------
A. A B. B C. C D. D
-------------------------------------------------------
【答案】A

In [85]:
len(failed_files) / len(examination_paper_list)

0.5170278637770898